# Optymalizacja trasy tramwajowej

Ten notebook zawiera wizualizację i optymalizację trasy tramwajowej w Krakowie.

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

import geopandas as gpd
from src.optimization.route_optimizer import RouteOptimizer, RouteConstraints
from src.visualization.route_visualizer import RouteVisualizer
from scripts.sourcing_data import TramData, OpenStreetMapData
from shapely.geometry import Point, LineString
import folium
import logging
import numpy as np
import random

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

INFO:scripts.sourcing_data:Logging initialized.


## Pobieranie danych

In [4]:
def load_data(data_dir: str) -> tuple[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]:
    """
    Wczytuje dane z plików GeoJSON.
    
    Args:
        data_dir: Katalog z danymi
        
    Returns:
        tuple[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]: 
            DataFrames z budynkami, ulicami, przystankami i liniami
    """
    
    buildings_path = os.path.join(data_dir, 'buildings.geojson')
    streets_path = os.path.join(data_dir, 'streets.geojson')
    stops_path = os.path.join(data_dir, 'stops.geojson')
    lines_path = os.path.join(data_dir, 'lines.geojson')
    
    logger.info("Wczytywanie danych z plików GeoJSON...")
    buildings_df = gpd.read_file(buildings_path)
    streets_df = gpd.read_file(streets_path)
    stops_df = gpd.read_file(stops_path)
    lines_df = gpd.read_file(lines_path)
    
    logger.info(f"Wczytano {len(buildings_df)} budynków")
    logger.info(f"Wczytano {len(streets_df)} ulic")
    logger.info(f"Wczytano {len(stops_df)} przystanków")
    logger.info(f"Wczytano {len(lines_df)} linii tramwajowych")
    return buildings_df, streets_df, stops_df, lines_df
   
data_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 'data')
buildings_df, streets_df, stops_df, lines_df = load_data(data_dir)

INFO:__main__:Wczytywanie danych z plików GeoJSON...
INFO:__main__:Wczytano 124332 budynków
INFO:__main__:Wczytano 423521 ulic
INFO:__main__:Wczytano 209 przystanków
INFO:__main__:Wczytano 21 linii tramwajowych


## Wizualizacja istniejących danych

In [ ]:
# Inicjalizacja wizualizatora
visualizer = RouteVisualizer(buildings_df, streets_df)

# Tworzenie podstawowej mapy
m = visualizer.create_base_map()

# Dodawanie istniejących linii
for _, row in lines_df.iterrows():
    coords = [(lat, lon) for lon, lat in row.geometry.coords]
    visualizer.plot_route(coords, m, f"Linia {row['line']}", color='gray')

# Dodawanie przystanków
for _, row in stops_df.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color='red',
        fill=True,
        popup=f'Przystanek {row.name}',
        name=f'Przystanek {row.name}'
    ).add_to(m)
    
m

## Optymalizacja trasy z wizualizacją procesu

In [ ]:
# Konfiguracja ograniczeń
constraints = RouteConstraints(
    min_distance_between_stops=200,  # 200m między przystankami
    max_distance_between_stops=1500,  # 1500m między przystankami
    max_angle=60,  # maksymalny kąt zakrętu
    min_route_length=3,  # minimalna liczba przystanków
    max_route_length=20,  # maksymalna liczba przystanków
    min_total_length=1000,  # minimalna długość trasy
    max_total_length=15000,  # maksymalna długość trasy
    min_distance_from_buildings=3  # minimalna odległość od budynków
)

# Inicjalizacja optymalizatora
optimizer = RouteOptimizer(
    buildings_df=buildings_df,
    streets_df=streets_df,
    stops_df=stops_df,
    lines_df=lines_df,
    constraints=constraints,
    population_size=1000,
    generations=500,
    mutation_rate=0.1,
    crossover_rate=0.8,
    min_stop_distance=200,  # 200 metrów
    max_stop_distance=1500,  # 1500 metrów
    population_weight=0.7,
    distance_weight=0.3
)

# Przygotowanie do wizualizacji
scores_history = []
best_route = None
best_score = float('-inf')

# Uruchomienie optymalizacji z wizualizacją
population = optimizer._create_initial_population()

for generation in range(optimizer.generations):
    # Ocena populacji
    scores = [optimizer._evaluate_route(route) for route in population]
    
    # Aktualizacja najlepszej trasy
    max_score_idx = np.argmax(scores)
    if scores[max_score_idx] > best_score:
        best_score = scores[max_score_idx]
        best_route = population[max_score_idx]
    
    scores_history.append(best_score)
    
    # Obliczenie granic obszaru
    bounds = (
        min(lon for lat, lon in best_route),
        min(lat for lat, lon in best_route),
        max(lon for lat, lon in best_route),
        max(lat for lat, lon in best_route)
    )
    
    # Generowanie mapy gęstości
    density_map = optimizer.density_calculator.get_density_map(
        grid_size=0.001,
        bounds=bounds
    )
    
    # Wizualizacja procesu
    visualizer.visualize_optimization_process(
        population=population,
        scores=scores,
        generation=generation + 1,
        best_route=best_route,
        best_score=best_score,
        bounds=bounds,
        density_map=density_map,
        update_interval=0.5
    )
    
    # Selekcja i tworzenie nowej populacji
    selected_indices = np.argsort(scores)[-optimizer.population_size//2:]
    selected = [population[i] for i in selected_indices]
    
    new_population = selected.copy()
    
    while len(new_population) < optimizer.population_size:
        parent1, parent2 = random.sample(selected, 2)
        child1, child2 = optimizer._crossover(parent1, parent2)
        
        child1 = optimizer._mutate(child1)
        child2 = optimizer._mutate(child2)
        
        if optimizer._is_valid_route(child1):
            new_population.append(child1)
        if optimizer._is_valid_route(child2) and len(new_population) < optimizer.population_size:
            new_population.append(child2)
    
    population = new_population

## Wizualizacja postępu optymalizacji

In [ ]:
visualizer.plot_optimization_progress(scores_history)